résumé de ce que j'ai fait:
génerer des données (sans le calcules de Moyenne Eligible Décision M E D) et sauvgarder en fichier data.txt
calcules de M E D sur un étudiant particulier
inserer les valeurs dans une ligne sur notre fichier (je pourrai eveuellement travailler sur le dataframe directement)


In [15]:
import pandas as pd
import numpy as np

# pour la partie 3
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.metrics import accuracy_score, classification_report

N_exemplaire = 100
file = "data"
arrond = 4
coeffs= [1, 1, 2, 0, 1] #calcul de la Moyenne
seuils = [11, 9.2, 11.23] # Calcul du status d'éligibilité (logique, BDD, PS)
thresholds = [12, 14] # prendre la décision

# 1 - Data generation

In [16]:
def generer_donnees(N_exemplaire=N_exemplaire, fichier=file, arrond=arrond):
    means = np.random.uniform(9, 15, size=8)
    stds = np.random.uniform(1, 3, size=8)
    column_names = ['S1', 'S2', 'S3', 'S4', 'S5', 'Logique', 'BDD', 'PS']

    data = {}
    data['N_candidat'] = range(0, N_exemplaire)
    for i, col_name in enumerate(column_names):
        values = np.random.normal(means[i], stds[i], N_exemplaire)
        if arrond > 0:
            values = np.round(values, arrond)
        elif arrond<0:
            raise ValueError
        else:
            values = values.astype(np.int32)
            
        data[col_name] = values
    data['Moyenne'] = np.nan
    data['Eligible'] = np.nan
    data['Decision'] = np.nan
    
    df = pd.DataFrame(data)
    path = f"{fichier}.txt"
    df.to_csv(path, index=False, sep='\t')
    return path

def save_df(df, file=file):
    path = f"{file}.txt"
    df.to_csv(path, index=False, sep='\t')
    return path

def upload_df(file=file):
    try:
        df = pd.read_csv(file, sep='\t')
        return df
    except FileNotFoundError:
        print("Fichier pas trouvé")

In [17]:
fichier = generer_donnees()
print("Fichier créé:", fichier)
df = upload_df(fichier)
display(df)

Fichier créé: data.txt


,N_candidat,S1,S2,S3,S4,S5,Logique,BDD,PS,Moyenne,Eligible,Decision
0,0,12.6975,5.5520,10.2587,11.4687,14.7828,8.8294,9.4861,6.6236,NaN,NaN,NaN
1,1,12.7541,8.8763,9.7158,10.7522,13.0602,13.6618,9.9581,9.1057,NaN,NaN,NaN
2,2,13.0767,9.9848,13.0219,12.0127,16.5487,12.0578,10.6973,10.5232,NaN,NaN,NaN
3,3,12.1845,7.0888,11.7462,9.7970,13.0640,9.1133,8.7637,9.2942,NaN,NaN,NaN
4,4,11.6004,9.5007,10.4854,9.1817,13.0674,9.5936,9.3448,11.1991,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
95,95,12.4583,7.3515,9.4859,11.8067,13.4084,14.3265,8.3256,8.4134,NaN,NaN,NaN
96,96,13.3641,12.1074,11.1748,11.3949,10.1441,4.8189,8.4519,10.2918,NaN,NaN,NaN
97,97,16.1169,9.7965,9.5595,10.9371,13.2612,7.2478,6.0559,10.6064,NaN,NaN,NaN
98,98,13.6526,5.8339,11.4225,12.0907,10.5158,8.3002,9.1476,7.9115,NaN,NaN,NaN


In [18]:
df.describe().loc[['mean', 'std']]

,N_candidat,S1,S2,S3,S4,S5,Logique,BDD,PS,Moyenne,Eligible,Decision
mean,49.500000,12.68926,9.530159,11.074252,10.888746,13.816722,10.177510,9.431475,9.083433,NaN,NaN,NaN
std,29.011492,1.50162,1.544119,1.205581,1.109769,2.149157,2.983154,1.523569,1.726551,NaN,NaN,NaN


# 2 -Calculs des valeurs manquantes en fur et à mesure ou bien au début..

In [19]:
def calcule_MED(S1, S2, S3, S4, S5, Logique, BDD, PS, coeffs=coeffs, seuils=seuils, thresholds2=thresholds):
    moyenne = (S1 * coeffs[0] + S2 * coeffs[1] + S3 * coeffs[2] + S4 * coeffs[3] + S5 * coeffs[4]) / sum(coeffs)
    eligible = Logique > seuils[0] and BDD > seuils[1] and PS > seuils[2]
    if eligible and moyenne > thresholds2[1]:
        decision = 'A'
    elif eligible and thresholds2[0] <= moyenne <= thresholds2[1]:
        decision = 'L'
    else:
        decision = 'R'
    return moyenne, eligible, decision

In [20]:
# c'est un test de calcul de M E D sur un étudiant

if df is not None:
    sample_data = df.iloc[0]
    moyenne, eligible, decision = calcule_MED(sample_data['S1'], sample_data['S2'], sample_data['S3'], sample_data['S4'], sample_data['S5'],
                                              sample_data['Logique'], sample_data['BDD'], sample_data['PS'], coeffs=[1, 1, 1, 1, 1], seuils=[12, 12, 12], thresholds2=[15, 17])

    print("Moyenne:", moyenne)
    print("Eligible:", eligible)
    print("Decision:", decision)
else:
    print("Error: DataFrame is None. Please check if the file exists or if there was an issue reading the file.")



Moyenne: 10.951939999999999
Eligible: False
Decision: R


In [21]:
# à partir d'un numéro de candidat, remplir  les données M  E D manquante dans la df.
def update_MED(N_candidat, df):
    row = df[df['N_candidat'] == N_candidat]
    if not row.empty:
        moyenne, eligible, decision = calcule_MED(row['S1'].values[0], row['S2'].values[0], row['S3'].values[0],
                                                   row['S4'].values[0], row['S5'].values[0], row['Logique'].values[0],
                                                   row['BDD'].values[0], row['PS'].values[0])
        df.loc[df['N_candidat'] == N_candidat, 'Moyenne'] = moyenne
        df.loc[df['N_candidat'] == N_candidat, 'Eligible'] = eligible
        df.loc[df['N_candidat'] == N_candidat, 'Decision'] = decision
        
        print(f"MED values for N_candidat {N_candidat} successfully updated in the dataframe")
    else:
        print(f"N_candidat {N_candidat} not found in the dataframe")


In [22]:
update_MED(3,df)


MED values for N_candidat 3 successfully updated in the dataframe


In [23]:
display(df)

,N_candidat,S1,S2,S3,S4,S5,Logique,BDD,PS,Moyenne,Eligible,Decision
0,0,12.6975,5.5520,10.2587,11.4687,14.7828,8.8294,9.4861,6.6236,NaN,NaN,NaN
1,1,12.7541,8.8763,9.7158,10.7522,13.0602,13.6618,9.9581,9.1057,NaN,NaN,NaN
2,2,13.0767,9.9848,13.0219,12.0127,16.5487,12.0578,10.6973,10.5232,NaN,NaN,NaN
3,3,12.1845,7.0888,11.7462,9.7970,13.0640,9.1133,8.7637,9.2942,11.16594,False,R
4,4,11.6004,9.5007,10.4854,9.1817,13.0674,9.5936,9.3448,11.1991,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
95,95,12.4583,7.3515,9.4859,11.8067,13.4084,14.3265,8.3256,8.4134,NaN,NaN,NaN
96,96,13.3641,12.1074,11.1748,11.3949,10.1441,4.8189,8.4519,10.2918,NaN,NaN,NaN
97,97,16.1169,9.7965,9.5595,10.9371,13.2612,7.2478,6.0559,10.6064,NaN,NaN,NaN
98,98,13.6526,5.8339,11.4225,12.0907,10.5158,8.3002,9.1476,7.9115,NaN,NaN,NaN


In [24]:
save_df(df)

'data.txt'

In [25]:
def compute_all_MED(df):
    for index, row in df.iterrows():
        N_candidat = row['N_candidat']
        moyenne, eligible, decision = calcule_MED(row['S1'], row['S2'], row['S3'], row['S4'], row['S5'],
                                                   row['Logique'], row['BDD'], row['PS'])
        df.loc[index, 'Moyenne'] = moyenne
        df.loc[index, 'Eligible'] = eligible
        df.loc[index, 'Decision'] = decision

    print("dataframe entièrement remplie")



In [26]:
display(df)
compute_all_MED(df)

,N_candidat,S1,S2,S3,S4,S5,Logique,BDD,PS,Moyenne,Eligible,Decision
0,0,12.6975,5.5520,10.2587,11.4687,14.7828,8.8294,9.4861,6.6236,NaN,NaN,NaN
1,1,12.7541,8.8763,9.7158,10.7522,13.0602,13.6618,9.9581,9.1057,NaN,NaN,NaN
2,2,13.0767,9.9848,13.0219,12.0127,16.5487,12.0578,10.6973,10.5232,NaN,NaN,NaN
3,3,12.1845,7.0888,11.7462,9.7970,13.0640,9.1133,8.7637,9.2942,11.16594,False,R
4,4,11.6004,9.5007,10.4854,9.1817,13.0674,9.5936,9.3448,11.1991,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
95,95,12.4583,7.3515,9.4859,11.8067,13.4084,14.3265,8.3256,8.4134,NaN,NaN,NaN
96,96,13.3641,12.1074,11.1748,11.3949,10.1441,4.8189,8.4519,10.2918,NaN,NaN,NaN
97,97,16.1169,9.7965,9.5595,10.9371,13.2612,7.2478,6.0559,10.6064,NaN,NaN,NaN
98,98,13.6526,5.8339,11.4225,12.0907,10.5158,8.3002,9.1476,7.9115,NaN,NaN,NaN


dataframe entièrement remplie


In [27]:
display(df)
save_df(df)
# now, vas-y regarde le fichier

,N_candidat,S1,S2,S3,S4,S5,Logique,BDD,PS,Moyenne,Eligible,Decision
0,0,12.6975,5.5520,10.2587,11.4687,14.7828,8.8294,9.4861,6.6236,10.70994,False,R
1,1,12.7541,8.8763,9.7158,10.7522,13.0602,13.6618,9.9581,9.1057,10.82444,False,R
2,2,13.0767,9.9848,13.0219,12.0127,16.5487,12.0578,10.6973,10.5232,13.13080,False,R
3,3,12.1845,7.0888,11.7462,9.7970,13.0640,9.1133,8.7637,9.2942,11.16594,False,R
4,4,11.6004,9.5007,10.4854,9.1817,13.0674,9.5936,9.3448,11.1991,11.02786,False,R
...,...,...,...,...,...,...,...,...,...,...,...,...
95,95,12.4583,7.3515,9.4859,11.8067,13.4084,14.3265,8.3256,8.4134,10.43800,False,R
96,96,13.3641,12.1074,11.1748,11.3949,10.1441,4.8189,8.4519,10.2918,11.59304,False,R
97,97,16.1169,9.7965,9.5595,10.9371,13.2612,7.2478,6.0559,10.6064,11.65872,False,R
98,98,13.6526,5.8339,11.4225,12.0907,10.5158,8.3002,9.1476,7.9115,10.56946,False,R


'data.txt'

In [28]:
df.describe().loc[['mean', 'std']]

,N_candidat,S1,S2,S3,S4,S5,Logique,BDD,PS,Moyenne
mean,49.500000,12.68926,9.530159,11.074252,10.888746,13.816722,10.177510,9.431475,9.083433,11.636929
std,29.011492,1.50162,1.544119,1.205581,1.109769,2.149157,2.983154,1.523569,1.726551,0.789655


# 3 - Model training and test

In [29]:
df = upload_df('data.txt')
X = df[['S1', 'S2', 'S3', 'S4', 'S5', 'Logique', 'BDD', 'PS']]
Y = df['Decision']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

model = RandomForestClassifier(random_state=42)
model.fit(X_train, Y_train)

Y_pred = model.predict(X_test)
accuracy = accuracy_score(Y_test, Y_pred)
report = classification_report(Y_test, Y_pred, zero_division=1)

print("Accuracy =", accuracy)
print("Classification Report:\n", report)

Accuracy = 1.0
Classification Report:
               precision    recall  f1-score   support

           R       1.00      1.00      1.00        20

    accuracy                           1.00        20
   macro avg       1.00      1.00      1.00        20
weighted avg       1.00      1.00      1.00        20



In [30]:
scores = cross_val_score(model, X, Y, cv=5)
Y_pred_cv = cross_val_predict(model, X, Y, cv=5)

accuracy = np.mean(scores)
report = classification_report(Y, Y_pred_cv, zero_division=1)

print("Cross-Validation Mean Accuracy:", accuracy)
print("Classification Report:\n", report)


c:\Users\yacin\anaconda3\Lib\site-packages\sklearn\model_selection\_split.py:725: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(
c:\Users\yacin\anaconda3\Lib\site-packages\sklearn\model_selection\_split.py:725: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(


Cross-Validation Mean Accuracy: 0.9800000000000001
Classification Report:
               precision    recall  f1-score   support

           L       1.00      0.00      0.00         2
           R       0.98      1.00      0.99        98

    accuracy                           0.98       100
   macro avg       0.99      0.50      0.49       100
weighted avg       0.98      0.98      0.97       100



In [31]:
def features_of_model(model):
    importances = model.feature_importances_
    indices = np.argsort(importances)[::-1]
    print("L'importance des features:")
    for f in range(X.shape[1]):
        print(f"    {X.columns[indices[f]]}: {importances[indices[f]]}")


features_of_model(model)
#je veux tester mon git

L'importance des features:
    PS: 0.280070307090137
    S2: 0.1878548918612361
    Logique: 0.10735073959465181
    S3: 0.10587725580545951
    BDD: 0.09845050182238399
    S1: 0.09347537860013433
    S4: 0.07393837572941939
    S5: 0.052982549496577734
